In [1]:
# aircraft_forecasting_optuna.py

# Importaciones básicas
import sys
import os
from pathlib import Path
import logging
import warnings
warnings.filterwarnings('ignore')

# Añadir el directorio padre al path
sys.path.append(str(Path().cwd().parent))

# Importar bibliotecas de análisis
import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_slice
)

# Importar módulos personalizados
from models import (
    ModelConfig,
    ATCAircraftDataLoader,
    AircraftDataPreprocessor,
    AircraftFeatureEngineer,
    ARIMAModel,
    ProphetModel,
    RandomForestModel,
    LSTMModel,
    EnsembleModel,
    AircraftForecaster
)

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuración
config = ModelConfig()
RANDOM_STATE = 42

In [2]:
def load_and_prepare_data(forecast_horizon=7):
    """Carga y prepara los datos para el entrenamiento."""
    logger.info("Cargando y preparando datos...")
    
    # 1. Cargar datos
    data_loader = ATCAircraftDataLoader(config)
    df = data_loader.get_training_data('daily_atc')
    
    # 2. Preprocesar datos
    preprocessor = AircraftDataPreprocessor(config)
    df_processed = preprocessor.preprocess_daily_data(df)
    
    # 3. Ingeniería de características
    feature_engineer = AircraftFeatureEngineer(config)
    df_featured = feature_engineer.create_features(df_processed)
    df_featured = feature_engineer.create_lagged_target(
        df_featured, 
        forecast_horizon=forecast_horizon
    )
    
    # 4. Preparar datos para modelado
    X, y = feature_engineer.select_features_for_model(df_featured)
    
    logger.info(f"Datos preparados: {len(X)} muestras, {len(X.columns)} características")
    
    return X, y, df_featured


In [3]:
def objective(trial, X, y):
    """
    Función objetivo para la optimización con Optuna.
    
    Args:
        trial: Objeto de prueba de Optuna
        X: Características de entrenamiento
        y: Variable objetivo
        
    Returns:
        Error de validación (MAE) a minimizar
    """
    # Selección del modelo
    algorithm = trial.suggest_categorical('algorithm', ['random_forest', 'prophet', 'lstm', 'arima'])

    # Crear una copia de la configuración
    trial_config = ModelConfig()

    if algorithm == 'random_forest':
        # Espacio de búsqueda para Random Forest
        trial_config.models['random_forest'] = {
            'n_estimators': trial.suggest_int('rf_n_estimators', 50, 500, step=50),
            'max_depth': trial.suggest_int('rf_max_depth', 3, 30),
            'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('rf_min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('rf_max_features', ['sqrt', 'log2', None]),
            'bootstrap': trial.suggest_categorical('rf_bootstrap', [True, False]),
            'random_state': RANDOM_STATE
        }
        
        model = RandomForestModel(trial_config)
        
    elif algorithm == 'prophet':
        # Espacio de búsqueda para Prophet
        trial_config.models['prophet'] = {
            'yearly_seasonality': trial.suggest_categorical('prophet_yearly', [True, False]),
            'weekly_seasonality': trial.suggest_categorical('prophet_weekly', [True, False]),
            'daily_seasonality': trial.suggest_categorical('prophet_daily', [True, False]),
            'changepoint_prior_scale': trial.suggest_float('prophet_changepoint_prior_scale', 0.001, 0.5, log=True),
            'seasonality_prior_scale': trial.suggest_float('prophet_seasonality_prior_scale', 0.1, 10, log=True),
            'seasonality_mode': trial.suggest_categorical('prophet_seasonality_mode', ['additive', 'multiplicative']),
            'changepoint_range': trial.suggest_float('prophet_changepoint_range', 0.8, 0.95),
            'n_changepoints': trial.suggest_int('prophet_n_changepoints', 10, 50, step=5)
        }
        
        model = ProphetModel(trial_config)
        
    elif algorithm == 'lstm':
        # Espacio de búsqueda para LSTM
        trial_config.models['lstm'] = {
            'sequence_length': trial.suggest_int('lstm_sequence_length', 7, 30, step=7),
            'hidden_units': trial.suggest_int('lstm_hidden_units', 32, 256, step=32),
            'dropout_rate': trial.suggest_float('lstm_dropout', 0.1, 0.5, step=0.1),
            'epochs': trial.suggest_int('lstm_epochs', 50, 200, step=50),
            'batch_size': trial.suggest_categorical('lstm_batch_size', [16, 32, 64]),
            'learning_rate': trial.suggest_float('lstm_learning_rate', 1e-4, 1e-2, log=True),
            'optimizer': trial.suggest_categorical('lstm_optimizer', ['adam', 'rmsprop']),
            'num_layers': trial.suggest_int('lstm_num_layers', 1, 3)
        }
        
        model = LSTMModel(trial_config)
        
    elif algorithm == 'arima':
        # Espacio de búsqueda para ARIMA
        p = trial.suggest_int('arima_p', 0, 5)
        d = trial.suggest_int('arima_d', 0, 2)
        q = trial.suggest_int('arima_q', 0, 5)
        P = trial.suggest_int('arima_P', 0, 3)
        D = trial.suggest_int('arima_D', 0, 2)
        Q = trial.suggest_int('arima_Q', 0, 3)
        s = 7  # Estacionalidad semanal
        
        trial_config.models['arima'] = {
            'order': (p, d, q),
            'seasonal_order': (P, D, Q, s)
        }
        
        model = ARIMAModel(trial_config)
    
    else:
        raise ValueError(f"Tipo de modelo no soportado: {algorithm}")
    
    # Validación cruzada temporal
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []
    
    for train_index, val_index in tscv.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Ajustar el modelo
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_val)

        # Alinear predicciones con el target y filtrar valores no finitos
        y_pred = np.asarray(y_pred).ravel()

        if len(y_pred) != len(y_val):
            min_len = min(len(y_pred), len(y_val))
            y_pred = y_pred[-min_len:]
            y_val_aligned = y_val.iloc[-min_len:]
        else:
            y_val_aligned = y_val

        valid_mask = np.isfinite(y_pred)
        if not valid_mask.any():
            raise ValueError("Predicciones no válidas: todas son NaN o infinitas")

        y_pred = y_pred[valid_mask]
        y_val_aligned = y_val_aligned.iloc[np.where(valid_mask)[0]]

        # Calcular métricas
        mae = mean_absolute_error(y_val_aligned, y_pred)
        scores.append(mae)
    
    # Devolver el MAE promedio
    return np.mean(scores)


In [4]:
def objective_emsemble(trial, X, y):
    """
    Función objetivo para la optimización con Optuna.
    
    Args:
        trial: Objeto de prueba de Optuna
        X: Características de entrenamiento
        y: Variable objetivo
        
    Returns:
        Error de validación (MAE) a minimizar
    """
    # Selección del modelo
    algorithm = trial.suggest_categorical('algorithm', ['ensemble'])

    # Crear una copia de la configuración
    trial_config = ModelConfig()
    
    # Configuración común para los modelos base
    rf_params = {
        'n_estimators': trial.suggest_int('rf_n_estimators', 50, 500, step=50),
        'max_depth': trial.suggest_int('rf_max_depth', 3, 30),
        'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('rf_min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('rf_max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('rf_bootstrap', [True, False]),
        'random_state': RANDOM_STATE
    }
    
    prophet_params = {
        'yearly_seasonality': trial.suggest_categorical('prophet_yearly', [True, False]),
        'weekly_seasonality': trial.suggest_categorical('prophet_weekly', [True, False]),
        'daily_seasonality': trial.suggest_categorical('prophet_daily', [True, False]),
        'changepoint_prior_scale': trial.suggest_float('prophet_changepoint_prior_scale', 0.001, 0.5, log=True),
        'seasonality_prior_scale': trial.suggest_float('prophet_seasonality_prior_scale', 0.1, 10, log=True),
        'seasonality_mode': trial.suggest_categorical('prophet_seasonality_mode', ['additive', 'multiplicative']),
        'changepoint_range': trial.suggest_float('prophet_changepoint_range', 0.8, 0.95),
        'n_changepoints': trial.suggest_int('prophet_n_changepoints', 10, 50, step=5)
    }
    
    lstm_params = {
        'sequence_length': trial.suggest_int('lstm_sequence_length', 7, 30, step=7),
        'hidden_units': trial.suggest_int('lstm_hidden_units', 32, 256, step=32),
        'dropout_rate': trial.suggest_float('lstm_dropout', 0.1, 0.5, step=0.1),
        'epochs': trial.suggest_int('lstm_epochs', 50, 200, step=50),
        'batch_size': trial.suggest_categorical('lstm_batch_size', [16, 32, 64]),
        'learning_rate': trial.suggest_float('lstm_learning_rate', 1e-4, 1e-2, log=True),
        'optimizer': trial.suggest_categorical('lstm_optimizer', ['adam', 'rmsprop']),
        'num_layers': trial.suggest_int('lstm_num_layers', 1, 3)
    }
    
    # Parámetros ARIMA
    p = trial.suggest_int('arima_p', 0, 5)
    d = trial.suggest_int('arima_d', 0, 2)
    q = trial.suggest_int('arima_q', 0, 5)
    P = trial.suggest_int('arima_P', 0, 3)
    D = trial.suggest_int('arima_D', 0, 2)
    Q = trial.suggest_int('arima_Q', 0, 3)
    s = 7  # Estacionalidad semanal

    if algorithm == 'random_forest':
        trial_config.models['random_forest'] = rf_params
        model = RandomForestModel(trial_config)
        
    elif algorithm == 'prophet':
        trial_config.models['prophet'] = prophet_params
        model = ProphetModel(trial_config)
        
    elif algorithm == 'lstm':
        trial_config.models['lstm'] = lstm_params
        model = LSTMModel(trial_config)
        
    elif algorithm == 'arima':
        trial_config.models['arima'] = {
            'order': (p, d, q),
            'seasonal_order': (P, D, Q, s)
        }
        model = ARIMAModel(trial_config)
        
    elif algorithm == 'ensemble':
        # Crear configuración para el ensemble
        trial_config = ModelConfig()
        
        # Definir espacio de búsqueda para los pesos
        weights = {
            'arima': trial.suggest_float('ensemble_weight_arima', 0, 1),
            'prophet': trial.suggest_float('ensemble_weight_prophet', 0, 1),
            'random_forest': trial.suggest_float('ensemble_weight_rf', 0, 1),
            'lstm': trial.suggest_float('ensemble_weight_lstm', 0, 1)
        }
        
        # Normalizar pesos para que sumen 1
        total = sum(weights.values())
        weights = {k: v/total for k, v in weights.items()}
        
        # Configurar pesos en la configuración
        trial_config.models['ensemble'] = {'weights': weights}
        
        # Configurar parámetros de los modelos base
        trial_config.models['random_forest'] = rf_params
        trial_config.models['prophet'] = prophet_params
        trial_config.models['lstm'] = lstm_params
        trial_config.models['arima'] = {
            'order': (p, d, q),
            'seasonal_order': (P, D, Q, s)
        }
        
        # Crear el ensemble y añadir modelos base
        ensemble = EnsembleModel(trial_config)
        ensemble.add_model(ARIMAModel(trial_config), weight=weights['arima'])
        ensemble.add_model(ProphetModel(trial_config), weight=weights['prophet'])
        ensemble.add_model(RandomForestModel(trial_config), weight=weights['random_forest'])
        ensemble.add_model(LSTMModel(trial_config), weight=weights['lstm'])
        
        model = ensemble
    
    else:
        raise ValueError(f"Tipo de modelo no soportado: {algorithm}")
    
    # Validación cruzada temporal
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []
    
    for train_index, val_index in tscv.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Ajustar el modelo
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_val)

        # Alinear predicciones con el target y filtrar valores no finitos
        y_pred = np.asarray(y_pred).ravel()

        if len(y_pred) != len(y_val):
            min_len = min(len(y_pred), len(y_val))
            y_pred = y_pred[-min_len:]
            y_val_aligned = y_val.iloc[-min_len:]
        else:
            y_val_aligned = y_val

        valid_mask = np.isfinite(y_pred)
        if not valid_mask.any():
            raise ValueError("Predicciones no válidas: todas son NaN o infinitas")

        y_pred = y_pred[valid_mask]
        y_val_aligned = y_val_aligned.iloc[np.where(valid_mask)[0]]

        # Calcular métricas
        mae = mean_absolute_error(y_val_aligned, y_pred)
        scores.append(mae)
    
    # Devolver el MAE promedio
    return np.mean(scores)


In [5]:
def optimize_hyperparameters(X, y, objective_fuc, n_trials=50):
    """
    Optimiza los hiperparámetros usando Optuna.
    
    Args:
        X: Características
        y: Variable objetivo
        n_trials: Número de pruebas a realizar
        
    Returns:
        study: Objeto de estudio de Optuna
    """
    # Crear o cargar estudio con almacenamiento persistente
    try:
        # Intentar cargar un estudio existente
        study = optuna.create_study(
            study_name=study_name,
            storage=storage_name,
            load_if_exists=True,
            direction='minimize',
            sampler=TPESampler(seed=RANDOM_STATE)
        )
        logger.info(f"Estudio cargado. Número de trials existentes: {len(study.trials)}")
    except Exception as e:
        # Si no existe, crear uno nuevo
        study = optuna.create_study(
            study_name=study_name,
            storage=storage_name,
            direction='minimize',
            sampler=TPESampler(seed=RANDOM_STATE)
        )
        logger.info("Nuevo estudio creado")
    
    # Función objetivo parcial
    def objective_wrapper(trial):
        return objective_fuc(trial, X, y)
    
    # Calcular cuántos trials nuevos necesitamos
    remaining_trials = max(0, n_trials - len(study.trials))
    
    if remaining_trials > 0:
        logger.info(f"Iniciando optimización con {remaining_trials} pruebas nuevas...")
        study.optimize(objective_wrapper, n_trials=remaining_trials, show_progress_bar=True)
    else:
        logger.info(f"Ya se han completado {len(study.trials)} trials. No se necesitan más pruebas.")
    
    # Mostrar resultados
    logger.info("\nResumen de la optimización:")
    logger.info(f"Número total de trials: {len(study.trials)}")
    logger.info(f"Mejor valor (MAE): {study.best_value:.4f}")
    logger.info("Mejores parámetros encontrados:")
    for key, value in study.best_params.items():
        logger.info(f"  {key}: {value}")
    
    return study


In [6]:
# Configuración de almacenamiento para Optuna
import sqlite3
from pathlib import Path

# Crear directorio para almacenamiento si no existe
storage_dir = Path("optuna_storage")
storage_dir.mkdir(exist_ok=True)

In [7]:
# Configurar el almacenamiento
storage_name = f"sqlite:///{storage_dir}/aircraft_forecasting_emsemble_params.db"
study_name = "aircraft_forecasting_study"

In [8]:
"""Función principal para ejecutar la optimización."""
# Cargar y preparar datos
X, y, _ = load_and_prepare_data(forecast_horizon=1)

study = optimize_hyperparameters(X, y, objective_emsemble, n_trials=100)

2026-01-04 22:50:13,269 - __main__ - INFO - Cargando y preparando datos...
2026-01-04 22:50:13,270 - models.data_loader - INFO - Cargando datos diarios ATC desde: /home/gabo/Personal/Universidad/04 - Cuarto Año/1er Semestre/Machine Learning/Proyectos/ml-project/models/data/ATC csvs/atc_dayatcopsummary_202512301506.csv
2026-01-04 22:50:13,285 - models.data_loader - INFO - Datos diarios cargados: 764 registros, columnas: ['arrivals', 'departures', 'overflights', 'nationals', 'unknown', 'total', 'fpp']
2026-01-04 22:50:13,286 - models.data_loader - INFO - Datos de entrenamiento preparados: 764 registros del 2022-12-11 00:00:00 al 2025-12-30 00:00:00
2026-01-04 22:50:13,287 - models.preprocessing - INFO - Iniciando preprocesamiento de datos diarios
2026-01-04 22:50:13,294 - models.preprocessing - INFO - Frecuencia diaria asegurada: 1116 días (original: 764)
2026-01-04 22:50:13,301 - models.preprocessing - INFO - Validación de integridad: OK
2026-01-04 22:50:13,302 - models.preprocessing - 

  0%|          | 0/45 [00:00<?, ?it/s]

2026-01-04 22:50:14,146 - models.model - INFO - Entrenando ensemble de modelos
2026-01-04 22:50:14,146 - models.model - INFO - Entrenando arima...
2026-01-04 22:50:14,193 - models.model - INFO - Entrenando ARIMA con order=(4, 0, 3), seasonal_order=(0, 1, 0, 7)
2026-01-04 22:50:14,363 - models.model - INFO - ARIMA entrenado exitosamente
2026-01-04 22:50:14,364 - models.model - INFO - Entrenando prophet...
2026-01-04 22:50:14,387 - models.model - INFO - Entrenando Prophet
2026-01-04 22:50:14,423 - cmdstanpy - INFO - Chain [1] start processing
2026-01-04 22:50:14,435 - cmdstanpy - INFO - Chain [1] done processing
2026-01-04 22:50:14,437 - models.model - INFO - Prophet entrenado exitosamente
2026-01-04 22:50:14,437 - models.model - INFO - Entrenando random_forest...
2026-01-04 22:50:14,438 - models.model - INFO - Entrenando Random Forest con 150 árboles
2026-01-04 22:50:14,631 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 22:50:14,643 - models.model - INFO - Entre

[I 2026-01-04 22:50:49,280] Trial 55 finished with value: 136.13084566065436 and parameters: {'algorithm': 'ensemble', 'rf_n_estimators': 150, 'rf_max_depth': 20, 'rf_min_samples_split': 3, 'rf_min_samples_leaf': 7, 'rf_max_features': None, 'rf_bootstrap': True, 'prophet_yearly': True, 'prophet_weekly': True, 'prophet_daily': False, 'prophet_changepoint_prior_scale': 0.4963027280372024, 'prophet_seasonality_prior_scale': 0.3494373559901818, 'prophet_seasonality_mode': 'additive', 'prophet_changepoint_range': 0.9311095806775519, 'prophet_n_changepoints': 30, 'lstm_sequence_length': 7, 'lstm_hidden_units': 32, 'lstm_dropout': 0.30000000000000004, 'lstm_epochs': 50, 'lstm_batch_size': 64, 'lstm_learning_rate': 0.0007924347570439766, 'lstm_optimizer': 'rmsprop', 'lstm_num_layers': 1, 'arima_p': 4, 'arima_d': 0, 'arima_q': 3, 'arima_P': 0, 'arima_D': 1, 'arima_Q': 0, 'ensemble_weight_arima': 0.21988978727561442, 'ensemble_weight_prophet': 0.09701081754903174, 'ensemble_weight_rf': 0.6986985

/home/gabo/Personal/Universidad/04 - Cuarto Año/1er Semestre/Machine Learning/Proyectos/ml-project/.venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
2026-01-04 22:50:50,243 - models.model - INFO - ARIMA entrenado exitosamente
2026-01-04 22:50:50,244 - models.model - INFO - Entrenando prophet...
2026-01-04 22:50:50,244 - models.model - INFO - Entrenando Prophet
2026-01-04 22:50:50,262 - cmdstanpy - INFO - Chain [1] start processing
2026-01-04 22:50:50,276 - cmdstanpy - INFO - Chain [1] done processing
2026-01-04 22:50:50,279 - models.model - INFO - Prophet entrenado exitosamente
2026-01-04 22:50:50,280 - models.model - INFO - Entrenando random_forest...
2026-01-04 22:50:50,281 - models.model - INFO - Entrenando Random Forest con 150 árboles
2026-01-04 22:50:50,441 - models.model - INFO - Random Forest entrenado exitosament

...

[I 2026-01-04 23:16:06,125] Trial 97 finished with value: 133.3146269477068 and parameters: {'algorithm': 'ensemble', 'rf_n_estimators': 250, 'rf_max_depth': 30, 'rf_min_samples_split': 2, 'rf_min_samples_leaf': 4, 'rf_max_features': 'log2', 'rf_bootstrap': True, 'prophet_yearly': True, 'prophet_weekly': False, 'prophet_daily': False, 'prophet_changepoint_prior_scale': 0.12904414347234439, 'prophet_seasonality_prior_scale': 0.5663572075802233, 'prophet_seasonality_mode': 'multiplicative', 'prophet_changepoint_range': 0.8729976635979296, 'prophet_n_changepoints': 20, 'lstm_sequence_length': 7, 'lstm_hidden_units': 32, 'lstm_dropout': 0.4, 'lstm_epochs': 50, 'lstm_batch_size': 64, 'lstm_learning_rate': 0.0030984835692436964, 'lstm_optimizer': 'rmsprop', 'lstm_num_layers': 1, 'arima_p': 4, 'arima_d': 0, 'arima_q': 3, 'arima_P': 0, 'arima_D': 1, 'arima_Q': 0, 'ensemble_weight_arima': 0.05199669820850325, 'ensemble_weight_prophet': 0.016479359384839583, 'ensemble_weight_rf': 0.3438368299448

2026-01-04 23:16:06,329 - models.model - INFO - Entrenando ensemble de modelos
2026-01-04 23:16:06,329 - models.model - INFO - Entrenando arima...
2026-01-04 23:16:06,329 - models.model - INFO - Entrenando ARIMA con order=(4, 0, 3), seasonal_order=(0, 1, 0, 7)
2026-01-04 23:16:06,498 - models.model - INFO - ARIMA entrenado exitosamente
2026-01-04 23:16:06,499 - models.model - INFO - Entrenando prophet...
2026-01-04 23:16:06,499 - models.model - INFO - Entrenando Prophet
2026-01-04 23:16:06,513 - cmdstanpy - INFO - Chain [1] start processing
2026-01-04 23:16:06,523 - cmdstanpy - INFO - Chain [1] done processing
2026-01-04 23:16:06,525 - models.model - INFO - Prophet entrenado exitosamente
2026-01-04 23:16:06,525 - models.model - INFO - Entrenando random_forest...
2026-01-04 23:16:06,526 - models.model - INFO - Entrenando Random Forest con 250 árboles
2026-01-04 23:16:06,739 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 23:16:06,760 - models.model - INFO - Entre

[I 2026-01-04 23:16:32,379] Trial 98 finished with value: 137.40432305850646 and parameters: {'algorithm': 'ensemble', 'rf_n_estimators': 250, 'rf_max_depth': 30, 'rf_min_samples_split': 2, 'rf_min_samples_leaf': 4, 'rf_max_features': 'log2', 'rf_bootstrap': True, 'prophet_yearly': True, 'prophet_weekly': False, 'prophet_daily': False, 'prophet_changepoint_prior_scale': 0.12782002042393506, 'prophet_seasonality_prior_scale': 0.5762004394494765, 'prophet_seasonality_mode': 'multiplicative', 'prophet_changepoint_range': 0.8715528676834217, 'prophet_n_changepoints': 20, 'lstm_sequence_length': 7, 'lstm_hidden_units': 32, 'lstm_dropout': 0.4, 'lstm_epochs': 50, 'lstm_batch_size': 64, 'lstm_learning_rate': 0.003025526975466707, 'lstm_optimizer': 'rmsprop', 'lstm_num_layers': 1, 'arima_p': 4, 'arima_d': 0, 'arima_q': 3, 'arima_P': 0, 'arima_D': 1, 'arima_Q': 0, 'ensemble_weight_arima': 0.0012099663804844105, 'ensemble_weight_prophet': 0.019556247881847982, 'ensemble_weight_rf': 0.32446965295

2026-01-04 23:16:32,585 - models.model - INFO - Entrenando ensemble de modelos
2026-01-04 23:16:32,585 - models.model - INFO - Entrenando arima...
2026-01-04 23:16:32,586 - models.model - INFO - Entrenando ARIMA con order=(4, 0, 3), seasonal_order=(0, 1, 0, 7)
2026-01-04 23:16:32,755 - models.model - INFO - ARIMA entrenado exitosamente
2026-01-04 23:16:32,755 - models.model - INFO - Entrenando prophet...
2026-01-04 23:16:32,756 - models.model - INFO - Entrenando Prophet
2026-01-04 23:16:32,770 - cmdstanpy - INFO - Chain [1] start processing
2026-01-04 23:16:32,782 - cmdstanpy - INFO - Chain [1] done processing
2026-01-04 23:16:32,784 - models.model - INFO - Prophet entrenado exitosamente
2026-01-04 23:16:32,784 - models.model - INFO - Entrenando random_forest...
2026-01-04 23:16:32,784 - models.model - INFO - Entrenando Random Forest con 250 árboles
2026-01-04 23:16:33,003 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 23:16:33,025 - models.model - INFO - Entre

[I 2026-01-04 23:16:59,307] Trial 99 finished with value: 131.29753751229822 and parameters: {'algorithm': 'ensemble', 'rf_n_estimators': 250, 'rf_max_depth': 30, 'rf_min_samples_split': 2, 'rf_min_samples_leaf': 4, 'rf_max_features': 'log2', 'rf_bootstrap': True, 'prophet_yearly': True, 'prophet_weekly': False, 'prophet_daily': False, 'prophet_changepoint_prior_scale': 0.08682327372138632, 'prophet_seasonality_prior_scale': 0.5422086783506253, 'prophet_seasonality_mode': 'multiplicative', 'prophet_changepoint_range': 0.8682473866293154, 'prophet_n_changepoints': 20, 'lstm_sequence_length': 7, 'lstm_hidden_units': 32, 'lstm_dropout': 0.4, 'lstm_epochs': 50, 'lstm_batch_size': 64, 'lstm_learning_rate': 0.002920132674289379, 'lstm_optimizer': 'rmsprop', 'lstm_num_layers': 1, 'arima_p': 4, 'arima_d': 0, 'arima_q': 3, 'arima_P': 0, 'arima_D': 1, 'arima_Q': 0, 'ensemble_weight_arima': 0.04691288600453619, 'ensemble_weight_prophet': 0.02694602357866513, 'ensemble_weight_rf': 0.32223054999611